# Veri Hazırlama

In [2]:
# "magic commands" to enable autoreload of your imported packages
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Amacımız, 9 `.csv` dosyasının tamamını tek bir sözlüğe (dict) `data` adında 9 `pandas.DataFrame` olarak yüklemektir. Burada:
- her **anahtar**, csv dosyasının **temizlenmiş adıdır**
- her **değer**, csv dosyasından oluşturulan **DataFrame**tir

```python
data = { 
    'sellers': DataFrame1,
    'orders': DataFrame2,
    ...
    }
```

### 1. `csv_path` değişkenini oluşturun, bu değişken `"csv" klasörü`nün yolunu bir string olarak depolayacaktır

Yolları kolayca yönetmek için Python'ın `pathlib` modülünü kullanalım. Pathlib çok kullanışlı bir `Path` sınıfı tanımlar.

Bir `Path` örneğini yolumuzu tanımlayan bir string vererek oluşturabiliriz.

In [3]:
from pathlib import Path
csv_path = Path("~/.workintech/olist/data/csv").expanduser()
csv_path

PosixPath('/Users/ilos/.workintech/olist/data/csv')

_`PosixPath`_? Bu, yolun Posix formatında olduğu anlamına gelir. POSIX (Portable Operating System Interface), Unix benzeri işletim sistemleri için uyumluluk ve birlikte çalışabilirliği sağlamak amacıyla standartlaştırılmış API'ler ve kuralların bir setidir. Hem Linux (örneğin WSL'deki Ubuntu) hem de macOS, POSIX uyumludur.

`expanduser()` kullanarak `~` işaretini ana klasörümüzün gerçek mutlak yoluna genişletiyoruz.

Şimdi klasörümüzdeki dosyaları listelemek için `iterdir()` metodunu kullanabiliriz:

In [4]:
file_paths = list(csv_path.iterdir())
file_paths

[PosixPath('/Users/ilos/.workintech/olist/data/csv/olist_sellers_dataset.csv'),
 PosixPath('/Users/ilos/.workintech/olist/data/csv/product_category_name_translation.csv'),
 PosixPath('/Users/ilos/.workintech/olist/data/csv/olist_orders_dataset.csv'),
 PosixPath('/Users/ilos/.workintech/olist/data/csv/olist_order_items_dataset.csv'),
 PosixPath('/Users/ilos/.workintech/olist/data/csv/olist_customers_dataset.csv'),
 PosixPath('/Users/ilos/.workintech/olist/data/csv/olist_geolocation_dataset.csv'),
 PosixPath('/Users/ilos/.workintech/olist/data/csv/olist_order_payments_dataset.csv'),
 PosixPath('/Users/ilos/.workintech/olist/data/csv/olist'),
 PosixPath('/Users/ilos/.workintech/olist/data/csv/olist_order_reviews_dataset.csv'),
 PosixPath('/Users/ilos/.workintech/olist/data/csv/olist_products_dataset.csv')]

In [5]:
# Kodunuzu aşağıda test edin. Dizindeki ilk csv dosyasını yüklemeyi deneyin
import pandas as pd
pd.read_csv(file_paths[0]).head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


### 2. csv dizinindeki tüm csv dosya adlarını içeren `file_names` listesini oluşturun

- Bunun böyle görünmesi gerekir: `file_names = ['olist_sellers_dataset.csv', ....]`
- `file_paths` adresinden başlayabilirsiniz
- Bir `Path` nesnesi `name` adında bir özelliğe sahiptir

In [7]:
file_names = [file_path.name for file_path in file_paths]

for name in file_names:
    print(name)

olist_sellers_dataset.csv
product_category_name_translation.csv
olist_orders_dataset.csv
olist_order_items_dataset.csv
olist_customers_dataset.csv
olist_geolocation_dataset.csv
olist_order_payments_dataset.csv
olist
olist_order_reviews_dataset.csv
olist_products_dataset.csv


### 3. Sözlük anahtarları listesi `key_names` oluşturun
file_names adresinden başlayarak:
- ".csv" sonekini (varsa) kaldırın
- "_dataset.csv" sonekini (varsa) kaldırın
- "olist_" önekini (varsa) kaldırın

<details>
    <summary>- İpucu - </summary>

- `.replace()`
    
- `str` diziler [ ] ile dilimlenebilecek yinelenebilir nesnelerdir
</details>

In [8]:
key_names = [
    name
    .replace('olist_', '')        # "olist_" ön ekini kaldır
    .replace('_dataset.csv', '')  # "_dataset.csv" son ekini kaldır
    .replace('.csv', '')          # Sadece ".csv" kaldıysa kaldır
    for name in file_names
]

print(key_names)

['sellers', 'product_category_name_translation', 'orders', 'order_items', 'customers', 'geolocation', 'order_payments', 'olist', 'order_reviews', 'products']


### 4. `data` sözlüğünü oluşturun

```python
data = { 
    'sellers': DataFrame1,
    'orders': DataFrame2,
    'order_items': DataFrame3,
    ...
    }
```
Burada `DataFrame1`, `DataFrame2`, ... gerçek `pandas.DataFrame` nesneleri olmalıdır.

<details>
    <summary>▸ İpucu</summary>

`zip()` metodu iki liste üzerinde yinelemek için çok kullanışlıdır
```python
for (x, y) in zip(['a','b','c'], [1,2,3]):
    print(x,y)

# ('a', 1), ('b', 2), ('c', 3) döndürür
    
```
</details>

In [10]:
import pandas as pd

data = {}

for key, path in zip(key_names, file_paths):
    # Yolun bir dosya olup olmadığını kontrol ediyoruz
    if path.is_file():
        data[key] = pd.read_csv(path)
    else:
        print(f"Atlanıyor (dosya değil): {path}")

# Sonucu kontrol edelim
print(f"Yüklenen tablo sayısı: {len(data)}")
print(data.keys())

Atlanıyor (dosya değil): /Users/ilos/.workintech/olist/data/csv/olist
Yüklenen tablo sayısı: 9
dict_keys(['sellers', 'product_category_name_translation', 'orders', 'order_items', 'customers', 'geolocation', 'order_payments', 'order_reviews', 'products'])


### 5. `olist/data.py` dosyasındaki `get_data()` metodunu uygulayın

Mantığımızı not defterinden `.py` dosyalarımıza taşımanın zamanı geldi. Bu, bu modül boyunca oluşturacağımız yeni not defterlerinde verileri kolayca yüklememize izin verecek.

Bu modülün klasöründe (bu not defterinin iki seviye yukarısında) `olist/data.py` dosyasını açın ve bu not defterine yazdığınız kodu `get_data()` metoduna taşımaya başlayın. Kodumuzu test etmek için yazdığımız satırları atlayın (`.py` dosyasında listeleri ve DataFrame'leri göstermeye gerek yok).

İşlev çağrıldığında DataFrames içeren sözlüğü döndürmelidir. Bunu csv dosyalarından biriyle deneyip görebilirsiniz:

In [11]:
from olist.data import Olist
Olist().get_data()['sellers'].head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


### Kodunuzu test edin

In [12]:
from nbresult import ChallengeResult
from olist.data import Olist
data = Olist().get_data()
result = ChallengeResult('get_data',
    keys_len=len(data),
    keys=sorted(list(data.keys())),
    columns=sorted(list(data['sellers'].columns)),
    vars_used=Olist.get_data.__code__.co_names
    )
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/ilos/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/ilos/01-Statistical-Inference/data-data-preparation/tests
plugins: anyio-4.8.0, typeguard-4.4.2
collecting ... collected 3 items

test_get_data.py::TestGetData::test_columns PASSED                       [ 33%]
test_get_data.py::TestGetData::test_keys PASSED                          [ 66%]
test_get_data.py::TestGetData::test_len PASSED                           [100%]

============================== 3 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/get_data.pickle

git commit -m 'Completed get_data step'

git push origin master



❓Bu kod parçası makinenizde her yerden çalışmalı, sadece bu not defterinden değil.
- Yeni bir terminal açın
- Ana klasörünüze gidin `cd`
- Bir `ipython` oturumu başlatın
- Yukarıdaki iki satır kodu test edin 👆

🏁 Tebrikler!

💾 Şunları commit ve push etmeyi unutmayın: 
* bu `data_preparation.ipynb` not defteri ve challenge klasöründeki `tests/get_data.pickle` test sonuçları
* ayrıca `/olist` klasöründeki `data.py` dosyası